# Create Table

In [0]:
CREATE SCHEMA IF NOT EXISTS pizza_runner;

DROP TABLE IF EXISTS runners;
CREATE TABLE runners (
  `runner_id` INTEGER,
  `registration_date` DATE
);
INSERT INTO runners
  (`runner_id`, `registration_date`)
VALUES
  (1, '2021-01-01'),
  (2, '2021-01-03'),
  (3, '2021-01-08'),
  (4, '2021-01-15');


DROP TABLE IF EXISTS customer_orders;
CREATE TABLE customer_orders (
  `order_id` INTEGER,
  `customer_id` INTEGER,
  `pizza_id` INTEGER,
  `exclusions` VARCHAR(4),
  `extras` VARCHAR(4),
  `order_time` TIMESTAMP
);

INSERT INTO customer_orders
  (`order_id`, `customer_id`, `pizza_id`, `exclusions`, `extras`, `order_time`)
VALUES
  ('1', '101', '1', '', '', '2020-01-01 18:05:02'),
  ('2', '101', '1', '', '', '2020-01-01 19:00:52'),
  ('3', '102', '1', '', '', '2020-01-02 23:51:23'),
  ('3', '102', '2', '', NULL, '2020-01-02 23:51:23'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '2', '4', '', '2020-01-04 13:23:46'),
  ('5', '104', '1', 'null', '1', '2020-01-08 21:00:29'),
  ('6', '101', '2', 'null', 'null', '2020-01-08 21:03:13'),
  ('7', '105', '2', 'null', '1', '2020-01-08 21:20:29'),
  ('8', '102', '1', 'null', 'null', '2020-01-09 23:54:33'),
  ('9', '103', '1', '4', '1, 5', '2020-01-10 11:22:59'),
  ('10', '104', '1', 'null', 'null', '2020-01-11 18:34:49'),
  ('10', '104', '1', '2, 6', '1, 4', '2020-01-11 18:34:49');


DROP TABLE IF EXISTS runner_orders;
CREATE TABLE runner_orders (
  `order_id` INTEGER,
  `runner_id` INTEGER,
  `pickup_time` VARCHAR(19),
  `distance` VARCHAR(7),
  `duration` VARCHAR(10),
  `cancellation` VARCHAR(23)
);

INSERT INTO runner_orders
  (`order_id`, `runner_id`, `pickup_time`, `distance`, `duration`, `cancellation`)
VALUES
  ('1', '1', '2020-01-01 18:15:34', '20km', '32 minutes', ''),
  ('2', '1', '2020-01-01 19:10:54', '20km', '27 minutes', ''),
  ('3', '1', '2020-01-03 00:12:37', '13.4km', '20 mins', NULL),
  ('4', '2', '2020-01-04 13:53:03', '23.4', '40', NULL),
  ('5', '3', '2020-01-08 21:10:57', '10', '15', NULL),
  ('6', '3', 'null', 'null', 'null', 'Restaurant Cancellation'),
  ('7', '2', '2020-01-08 21:30:45', '25km', '25mins', 'null'),
  ('8', '2', '2020-01-10 00:15:02', '23.4 km', '15 minute', 'null'),
  ('9', '2', 'null', 'null', 'null', 'Customer Cancellation'),
  ('10', '1', '2020-01-11 18:50:20', '10km', '10minutes', 'null');


DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names (
  `pizza_id` INTEGER,
  `pizza_name` STRING
);
INSERT INTO pizza_names
  (`pizza_id`, `pizza_name`)
VALUES
  (1, 'Meatlovers'),
  (2, 'Vegetarian');


DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes (
  `pizza_id` INTEGER,
  `toppings` STRING
);
INSERT INTO pizza_recipes
  (`pizza_id`, `toppings`)
VALUES
  (1, '1, 2, 3, 4, 5, 6, 8, 10'),
  (2, '4, 6, 7, 9, 11, 12');


DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings (
  `topping_id` INTEGER,
  `topping_name` STRING
);
INSERT INTO pizza_toppings
  (`topping_id`, `topping_name`)
VALUES
  (1, 'Bacon'),
  (2, 'BBQ Sauce'),
  (3, 'Beef'),
  (4, 'Cheese'),
  (5, 'Chicken'),
  (6, 'Mushrooms'),
  (7, 'Onions'),
  (8, 'Pepperoni'),
  (9, 'Peppers'),
  (10, 'Salami'),
  (11, 'Tomatoes'),
  (12, 'Tomato Sauce');

# Clean Table

## Clean 'runner_orders' Table

In [0]:
DROP TABLE IF EXISTS runner_orders_cleaned;

CREATE TABLE runner_orders_cleaned (
  order_id INTEGER PRIMARY KEY,
  runner_id INTEGER,
  pickup_time STRING,
  distance FLOAT,
  duration FLOAT,
  cancellation STRING
);

INSERT INTO runner_orders_cleaned
SELECT 
  order_id,
  runner_id,
  NULLIF(pickup_time, 'null') AS pickup_time,
  CASE 
      WHEN TRIM(distance) = 'null' OR distance IS NULL THEN NULL
      ELSE ROUND(CAST(REGEXP_REPLACE(distance, '[^0-9.]', '') AS FLOAT), 2)
  END AS distance,
  CASE
      WHEN TRIM(duration) = 'null' OR duration IS NULL THEN NULL 
      ELSE CAST(REGEXP_REPLACE(duration, '[^0-9.]', '') AS FLOAT)
  END AS duration,
  CASE
      WHEN TRIM(cancellation) IN ('Restaurant Cancellation', 'Customer Cancellation')
      THEN TRIM(cancellation)
      ELSE NULL
  END AS cancellation
FROM runner_orders;

## Clean 'customer_orders' Table

In [0]:
DROP TABLE IF EXISTS customer_orders_cleaned;

CREATE TABLE  customer_orders_cleaned(
  order_id INT,
  customer_id INT,
  pizza_id INT,
  exclusions STRING,
  extras STRING,
  order_date TIMESTAMP
);

INSERT INTO customer_orders_cleaned 
SELECT 
  order_id,
  customer_id,
  pizza_id,
  CASE
      WHEN exclusions = '' OR TRIM(exclusions) = 'null' THEN NULL
      ELSE exclusions
  END AS exclusions,
  CASE 
      WHEN extras = '' OR extras = 'null' THEN NULL
      ELSE extras
  END AS extras,
  order_time
FROM customer_orders;

# Business Questions

## A. Pizza Metrics

### Q1.How many pizzas were ordered?

In [0]:
SELECT 
  COUNT(pizza_id) AS total_pazzas
FROM customer_orders_cleaned;

### Q2. How many unique customer orders were made?

In [0]:
SELECT
    COUNT(DISTINCT order_id) AS unique_orders
FROM customer_orders_cleaned;

### Q3. How many successful orders were delivered by each runner?

In [0]:
SELECT 
    runner_id,
    COUNT(*) AS successfully_delivered_by_runner
FROM runner_orders_cleaned
WHERE pickup_time IS NOT NULL
GROUP BY runner_id;

### Q4. How many of each type of pizza was delivered?

In [0]:
SELECT 
    pizza_name,
    COUNT(*) total_order
FROM customer_orders_cleaned AS co
JOIN pizza_names AS pn 
    ON pn.pizza_id = co.pizza_id
JOIN runner_orders_cleaned AS ro
    ON ro.order_id = co.order_id
WHERE pickup_time IS NOT NULL
GROUP BY pizza_name;

### Q5. How many Vegetarian and Meatlovers were ordered by each customer?

In [0]:
SELECT
    customer_id,
    SUM(CASE WHEN pizza_name = 'Vegetarian' THEN 1 ELSE 0 END) AS vegetarian,
    SUM(CASE WHEN pizza_name = 'Meatlovers' THEN 1 ELSE 0 END) AS meatlovers
FROM customer_orders_cleaned AS co
JOIN pizza_names AS pn 
    ON co.pizza_id = pn.pizza_id
GROUP BY customer_id
ORDER BY customer_id;

### Q6. What was the maximum number of pizzas delivered in a single order?

In [0]:
SELECT 
    co.order_id,
    COUNT(*) AS max_delivery_perOrder
FROM customer_orders_cleaned AS co
JOIN runner_orders_cleaned AS ro ON co.order_id = ro.order_id
WHERE pickup_time IS NOT NULL
GROUP BY co.order_id
ORDER BY COUNT(*) DESC;

### Q7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [0]:
SELECT
    customer_id,
    SUM(
        CASE 
            WHEN exclusions IS NOT NULL OR extras IS NOT NULL THEN 1 
            ELSE 0 
        END
    ) AS CHANGES,
    SUM(
        CASE 
            WHEN exclusions IS NULL AND extras IS NULL THEN 1 
            ELSE 0 
        END
    ) AS NO_CHANGES

FROM customer_orders_cleaned
GROUP BY customer_id;


### Q8. How many pizzas were delivered that had both exclusions and extras?

In [0]:
SELECT 
    COUNT(*) AS total_delivered
FROM customer_orders_cleaned AS co
LEFT JOIN runner_orders_cleaned AS ro ON co.order_id = ro.order_id
WHERE pickup_time IS NOT NULL AND co.exclusions IS NOT NULL AND co.extras IS NOT NULL;


### Q9. What was the total volume of pizzas ordered for each hour of the day?

In [0]:
SELECT 
    HOUR(order_date) AS orderd_hour,
    COUNT(*) AS total_orders
FROM customer_orders_cleaned
GROUP BY HOUR(order_date);

### Q10. What was the volume of orders for each day of the week?

In [0]:
SELECT 
    DATE_PART('dow', order_date) AS Day,
    DATE_FORMAT(order_date, 'EEEE') AS Day_Name,
    COUNT(*) AS total_orders
FROM customer_orders_cleaned
GROUP BY DATE_PART('dow', order_date),
         DATE_FORMAT(order_date, 'EEEE')
ORDER BY DATE_PART('dow', order_date);

## B. Runner and Customer Experience

### Q1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [0]:
SELECT 

    DATE_ADD(
        DATE '2021-01-01',
        CAST(
            FLOOR(
                DATEDIFF(registration_date, DATE '2021-01-01')
                / 7) * 7 
                AS INT)
    ) AS week_start,

    COUNT(*) AS total

FROM runner_orders_cleaned AS ro
JOIN runners AS r ON ro.runner_id = r.runner_id

GROUP BY  DATE_ADD(
        DATE '2021-01-01',
        CAST(FLOOR(DATEDIFF(registration_date, DATE '2021-01-01') / 7) * 7 AS INT)
    );

### Q2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [0]:
SELECT 
    ro.runner_id,
    AVG(
        TIMESTAMPDIFF(MINUTE, 
            TRY_CAST(co.order_date AS TIMESTAMP), 
            TRY_CAST(ro.pickup_time AS TIMESTAMP)
            )
        ) AS average_time_to_pickup
FROM customer_orders_cleaned AS co
JOIN runner_orders_cleaned AS ro
ON co.order_id = ro.order_id
WHERE co.order_date IS NOT NULL AND 
      TRY_CAST(ro.pickup_time AS TIMESTAMP) IS NOT NULL
GROUP BY ro.runner_id;

### Q3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [0]:
SELECT
  number_of_pizza,
  AVG(preparation_time)
FROM(
    SELECT 
      co.order_id,
      MAX(
        TIMESTAMPDIFF(
            MINUTE, 
            TRY_CAST(co.order_date AS TIMESTAMP), 
            TRY_CAST(ro.pickup_time AS TIMESTAMP)
            )
        ) AS preparation_time,
        COUNT(co.pizza_id) AS number_of_pizza
    FROM customer_orders_cleaned AS co
    JOIN runner_orders_cleaned AS ro
    ON co.order_id = ro.order_id
    WHERE co.order_date IS NOT NULL AND 
          TRY_CAST(ro.pickup_time AS TIMESTAMP) IS NOT NULL
    GROUP BY co.order_id
) AS temp_table
GROUP BY number_of_pizza

### Q4. What was the average distance travelled for each customer?

In [0]:
SELECT 
    co.customer_id,
    ROUND(AVG(ro.distance), 2) AS Avg_distance
FROM customer_orders_cleaned AS co
JOIN runner_orders_cleaned AS ro
    ON co.order_id = ro.order_id
WHERE ro.distance IS NOT NULL
      AND ro.cancellation IS NULL
GROUP BY co.customer_id
ORDER BY ROUND(AVG(ro.distance), 2) DESC;

### Q5. What was the difference between the longest and shortest delivery times for all orders?

In [0]:
SELECT 
  (MAX(duration) - MIN(duration)) AS diff_longest_shortest_delivery
FROM runner_orders_cleaned;

### Q6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

In [0]:
SELECT 
  runner_id,
  order_id,
  ROUND(AVG(distance / duration), 2) AS speed
FROM runner_orders_cleaned
WHERE pickup_time IS NOT NULL
GROUP BY runner_id, order_id
ORDER BY runner_id,order_id;

### Q7. What is the successful delivery percentage for each runner?

In [0]:
SELECT 
    runner_id,
    ROUND(
        100.0 * 
        SUM(
            CASE 
                WHEN pickup_time IS NOT NULL THEN 1
                ELSE 0
            END 
        ) / COUNT(*) , 2)
        AS successful_delivery_percentage
FROM runner_orders_cleaned
GROUP BY runner_id;

## C. Ingredient Optimisation

### Q1. What are the standard ingredients for each pizza?

In [0]:
SELECT
    pn.pizza_id,
    pt.topping_name
FROM pizza_names AS pn
JOIN pizza_recipes AS pr ON pn.pizza_id = pr.pizza_id
JOIN (
    SELECT
        pizza_id,
        explode(split(toppings, ',')) AS topping_id
    FROM pizza_recipes
) AS exploded ON pr.pizza_id = exploded.pizza_id
JOIN pizza_toppings AS pt ON pt.topping_id = CAST(exploded.topping_id AS INT);

### Q2. What was the most commonly added extra?

In [0]:
SELECT 
  topping_name,
  COUNT(*) AS total
FROM (
  SELECT 
    pizza_id, 
    explode(split(extras, ',')) AS extras
  FROM customer_orders_cleaned
  WHERE extras IS NOT NULL
) t
JOIN pizza_toppings AS pt
  ON pt.topping_id = t.extras
GROUP BY topping_name
ORDER BY total DESC
LIMIT 1;

### Q3. What was the most common exclusion?

In [0]:
SELECT
  topping_name,
  COUNT(*) AS total
FROM(
  SELECT 
    co.pizza_id,
    explode(split(exclusions, ',')) AS exclusions
  FROM customer_orders_cleaned AS co
  WHERE exclusions IS NOT NULL
) AS t
JOIN pizza_toppings AS pt
  ON pt.topping_id = t.exclusions
GROUP BY topping_name
ORDER BY total DESC
LIMIT 1;

### Q4. Generate an order item for each record in the customers_orders table in the format of one of the following:
    Meat Lovers
    Meat Lovers - Exclude Beef
    Meat Lovers - Extra Bacon
    Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

In [0]:
WITH base AS (
  SELECT 
    co.order_id,
    co.exclusions,
    co.extras,
    pn.pizza_name
  FROM customer_orders_cleaned AS co
  JOIN pizza_names AS pn
    ON pn.pizza_id = co.pizza_id
),

excluded AS(
  SELECT
      exc.order_id,
      concat_ws(', ', collect_list(pt.topping_name)) AS excluded_toppings
  FROM(
    SELECT 
      order_id,
      explode(split(exclusions, ',')) AS exclusions
    FROM base
    WHERE exclusions IS NOT NULL
  ) AS exc
  JOIN pizza_toppings AS pt
      ON pt.topping_id = exc.exclusions
  GROUP BY order_id
),

extra AS (
  SELECT
    xtra.order_id,
    concat_ws(', ', collect_list(pt.topping_name)) AS extra_toppings
  FROM(
    SELECT
      order_id,
      explode(split(extras, ',')) AS extras
  FROM base
  WHERE extras IS NOT NULL
  ) xtra
  JOIN pizza_toppings AS pt
      ON pt.topping_id = xtra.extras
  GROUP BY xtra.order_id
)

SELECT 
  b.order_id,
  CASE 
    WHEN 
      excluded_toppings IS NOT NULL AND
      extra_toppings IS NOT NULL
    THEN concat(b.pizza_name, ' - Excluded ', exc.excluded_toppings, ' - Extra ', xtra.extra_toppings)
    WHEN 
        excluded_toppings IS NOT NULL
    THEN concat(b.pizza_name, ' - Excluded ', exc.excluded_toppings)
    WHEN 
        extra_toppings IS NOT NULL
    THEN 
        concat(b.pizza_name, ' - Extra ', xtra.extra_toppings)
    ELSE
        b.pizza_name
  END AS order_item
FROM base AS b
LEFT JOIN excluded AS exc
  ON exc.order_id = b.order_id
LEFT JOIN extra AS xtra
  ON xtra.order_id = b.order_id;

### Q5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients
    For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

In [0]:
WITH base AS (
  SELECT 
    co.order_id,
    co.pizza_id,
    co.extras,
    co.exclusions,
    pn.pizza_name,
    pr.toppings
  FROM customer_orders_cleaned AS co
  JOIN pizza_names AS pn
    ON co.pizza_id = pn.pizza_id
  JOIN pizza_recipes AS pr
    ON co.pizza_id = pr.pizza_id
),
-- explode default toppings
default_topping AS (
  SELECT 
    b.order_id,
    CAST(TRIM(topping_id) AS INT) AS topping_id,
    1 AS qty
  FROM base AS b
  LATERAL VIEW explode(split(toppings, ',')) AS topping_id
),

-- explode exclusions
excluded_topping AS (
    SELECT 
     b.order_id,
     CAST(TRIM(exclusion) AS INT) AS topping_id,
     -1 AS qty
    FROM base AS b
    LATERAL VIEW explode(split(exclusions, ',')) AS exclusion
),

-- explode extras
extra_topping AS (
  SELECT 
    b.order_id,
    CAST(TRIM(extra) AS INT) AS topping_id,
    1 AS qty
  FROM base AS b
  LATERAL VIEW explode(split(extras, ',')) AS extra
),

-- combine exploded default toppings, exploded extras, exploded exclusions
combined AS (
  SELECT * FROM default_topping
  UNION ALL
  SELECT * FROM excluded_topping
  UNION ALL
  SELECT * FROM extra_topping
),

-- calculate final quantities
final_qty AS (
  SELECT 
    c.order_id,
    c.topping_id,
    SUM(c.qty) AS final_qty
  FROM combined AS c
  GROUP BY c.order_id,
           c.topping_id
  HAVING SUM(c.qty) > 0
)

SELECT
  b.order_id,
  CONCAT(
    pizza_name, ': ',
    concat_ws(
      ', ', 
      collect_list(
        CASE
          WHEN f.final_qty > 1
              THEN CONCAT (f.final_qty, 'x ', pt.topping_name)
          ELSE pt.topping_name
        END
      )
    )
  ) AS ingredient_list
  FROM final_qty AS f
  JOIN pizza_toppings AS pt ON pt.topping_id = f.topping_id
  JOIN base AS b ON b.order_id = f.order_id
  GROUP BY b.order_id, b.pizza_name
  ORDER BY b.order_id;

### Q6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

In [0]:
WITH base AS (
    SELECT
        co.order_id,
        pr.toppings,
        co.exclusions,
        co.extras
    FROM customer_orders_cleaned co
    JOIN runner_orders_cleaned ro
        ON co.order_id = ro.order_id
    JOIN pizza_recipes pr
        ON co.pizza_id = pr.pizza_id
    WHERE ro.pickup_time IS NOT NULL

),
-- explode toppings
default_topping AS (
  SELECT 
    order_id,
    CAST(TRIM(topping) AS INT) topping_id,
    1 AS qty
  FROM base 
  LATERAL VIEW explode(split(toppings, ',')) AS topping
),

-- explode extra
extra_topping AS (
  SELECT 
    order_id,
    CAST(TRIM(extra) AS INT) topping_id,
    1 AS qty
  FROM base
  LATERAL VIEW explode(split(extras, ',')) AS extra
),

-- explode exclusions
excluded_toppings AS (
  SELECT 
    order_id,
    CAST(TRIM(exclusion) AS INT) AS topping_id,
    -1 AS qty
  FROM base
  LATERAL VIEW explode(split(exclusions, ',')) AS exclusion
),

combined AS (
  SELECT * FROM default_topping
  UNION ALL
  SELECT * FROM extra_topping
  UNION ALL
  SELECT * FROM excluded_toppings
)
SELECT
    pt.topping_name,
    SUM(qty) AS total_quantity
FROM combined c
JOIN pizza_toppings pt
    ON pt.topping_id = c.topping_id
GROUP BY pt.topping_name
ORDER BY total_quantity DESC;

## D. Pricing and Ratings

### Q1. If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

In [0]:
SELECT
  SUM(
      CASE
        WHEN  pizza_name = 'Meatlovers' THEN 12
        ELSE 10
      END 
      ) AS total
FROM customer_orders_cleaned AS co
JOIN runner_orders_cleaned AS ro
  ON co.order_id = ro.order_id
JOIN pizza_names AS pn
  ON pn.pizza_id = co.pizza_id
WHERE ro.pickup_time IS NOT NULL;

### Q2. What if there was an additional $1 charge for any pizza extras?
    Add cheese is $1 extra

In [0]:
WITH base AS (
  SELECT 
    co.order_id,
    pn.pizza_name,
    co.extras
  FROM customer_orders_cleaned co
  JOIN runner_orders_cleaned ro
    ON co.order_id = ro.order_id
  JOIN pizza_names pn
    ON co.pizza_id = pn.pizza_id
  WHERE ro.pickup_time IS NOT NULL
),

extra_count AS (
  SELECT
    order_id,
    COUNT(*) AS extra_cnt
  FROM base
  LATERAL VIEW explode(split(extras, ',')) t AS extra
  WHERE extras IS NOT NULL
  GROUP BY order_id
)

SELECT
  b.order_id,
  b.pizza_name,
  CASE
    WHEN b.pizza_name = 'Meatlovers' THEN 12
    ELSE 10
  END + COALESCE(e.extra_cnt, 0) AS final_price
FROM base b
LEFT JOIN extra_count AS e
  ON b.order_id = e.order_id
ORDER BY b.order_id;


### Q3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

In [0]:
DROP TABLE IF EXISTS runner_ratings;
CREATE TABLE runner_ratings (
  rating_id INT,
  order_id INT,
  runner_id INT,
  rating INT CHECK (rating BETWEEN 1 AND 5),
  review STRING,
  rating_time TIMESTAMP
);

INSERT INTO runner_ratings VALUES
(1, 1, 1, 5, 'Fast delivery', current_timestamp()),
(2, 2, 1, 4, 'Pretty quick', current_timestamp()),
(3, 3, 2, 3, 'Average service', current_timestamp()),
(4, 4, 3, 5, 'Excellent runner', current_timestamp()),
(5, 5, 2, 2, 'Late delivery', current_timestamp());

### Q4. Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?
      customer_id
      order_id
      runner_id
      rating
      order_time
      pickup_time
      Time between order and pickup
      Delivery duration
      Average speed
      Total number of pizzas

In [0]:
SELECT
  co.customer_id,
  ro.order_id,
  ro.runner_id,
  rr.rating,
  co.order_date,
  ro.pickup_time,
  TIMESTAMPDIFF(MINUTE, co.order_date, ro.pickup_time) AS time_to_pickup,
  ro.duration AS delivery_duration,
  ROUND(ro.distance / ro.duration, 2) AS avg_speed,
  COUNT(co.pizza_id) AS total_pizzas
FROM runner_orders_cleaned ro
JOIN customer_orders_cleaned co
  ON ro.order_id = co.order_id
LEFT JOIN runner_ratings rr
  ON ro.order_id = rr.order_id
 AND ro.runner_id = rr.runner_id
WHERE ro.pickup_time IS NOT NULL
GROUP BY
  co.customer_id,
  ro.order_id,
  ro.runner_id,
  rr.rating,
  co.order_date,
  ro.pickup_time,
  ro.duration,
  ro.distance
ORDER BY ro.order_id;


### Q5. If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

In [0]:
WITH revenue AS (
  SELECT
    SUM(
      CASE
        WHEN pn.pizza_name = 'Meat Lovers' THEN 12
        ELSE 10
      END
    ) AS total_revenue
  FROM customer_orders_cleaned co
  JOIN runner_orders_cleaned ro
    ON co.order_id = ro.order_id
  JOIN pizza_names pn
    ON co.pizza_id = pn.pizza_id
  WHERE ro.pickup_time IS NOT NULL
),

costs AS (
  SELECT
    SUM(distance * 0.30) AS total_cost
  FROM runner_orders_cleaned
  WHERE pickup_time IS NOT NULL
)

SELECT
  ROUND(r.total_revenue - c.total_cost, 2) AS profit
FROM revenue r
CROSS JOIN costs c;
